In [1]:
import pandas as pd
import numpy as np
import joblib
import time
import collections
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix, mean_squared_error, log_loss, accuracy_score


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
pd.__version__

'0.24.2'

In [3]:
df = pd.read_csv('TenantInfo-and-usage_shuffled_inf.csv', nrows=1000
                )

In [4]:
df.shape

(1000, 228)

In [5]:
df.head()

,TenantId,CreatedDate,CreateDateOfFirstSubscription,CountryCode,AllupSeats,EXOSubscriptionsCount,OD4BSubscriptionsCount,SfBSubscriptionsCount,TeamsSubscriptionsCount,PaidCount,ProjectSubscriptionsCount,SPOSubscriptionsCount,ActivatedSubscriptionTotalCount,VisioSubscriptionsCount,TrialSubscriptionsCount,NonTrialSubscriptionsCount,Languange,DataCenterInstance,DataCenterModel,HasEXO,HasSPO,HasOD4B,HasSfB,HasYammer,HasTeams,HasTeamsFreemium,HasKaizala,HasProPlus,HasAADP,HasAIP,HasAATP,HasIntune,HasMCAS,HasWDATP,HasAudioConference,HasPhoneSystem,HasEdiscovery,HasCompliance,HasThreatIntelligence,HasCustomerLockbox,HasOATP,HasAADPP2,HasAIPP2,HasWindows,HasO365CAS,HasCASDiscovery,HasPAM,HasPowerBI,HasPowerBIPremium,HasPowerBIPro,HasVisio,HasProject,HasNonTrial,HasSubscription_AllCounted,WithoutTenantAdmin,OrgNameEnteredInCommerce,HasPartnerTenants,SignupLocationInfo_Country,SignupLocationInfo_CountryCode,SignupLocationInfo_Region,Subscriptions_HasPaidSeats,TopParents_AreaName,TopParents_BigAreaName,TopParents_CountryCode,TopParents_Industry,TopParents_RegionName,TopParents_SegmentGroup,TopParents_SubRegionName,TopParents_VerticalName,EXOEnabledUsers,SPOEnabledUsers,OD4BEnabledUsers,SFBEnabledUsers,TeamEnabledUsers,YammerEnabledUsers,PPDEnabledUsers,KaizalaEnabledUsers,AADPEnabledUsers,AIPEnabledUsers,AATPEnabledUsers,IntuneEnabledUsers,MCASEnabledUsers,WDATPEnabledUsers,AudioConferenceEnabledUsers,PhoneSystemEnabledUsers,EdiscoveryEnabledUsers,ComplianceEnabledUsers,ThreatIntelligenceEnabledUsers,CustomerLockboxEnabledUsers,OATPEnabledUsers,AADPP2EnabledUsers,AIPP2EnabledUsers,WindowsEnabledUsers,O365CASEnabledUsers,CASDiscoveryEnabledUsers,PAMEnabledUsers,O365EnabledUsers,EMSEnabledUsers,M365EnabledUsers,O365E5EnabledUsers,EMSE5EnabledUsers,M365E5EnabledUsers,TotalUsers,PaidEXOSeats,PaidSPOSeats,PaidOD4BSeats,PaidYammerSeats,PaidTeamsSeats,PaidSFBSeats,PaidKaizalaSeats,PaidProplusSeats,PaidAADPSeats,PaidAIPSeats,PaidAATPSeats,PaidIntuneSeats,PaidMCASSeats,PaidWDATPSeats,PaidAudioConferenceSeats,PaidPhoneSystemSeats,PaidOATPSeats,PaidAADPP2Seats,PaidAIPP2Seats,PaidWindowsSeats,PaidO365CASSeats,PaidCASDiscoverySeats,PaidPAMSeats,PaidPowerBISeats,PaidPowerBIPremiumSeats,PaidPowerBIProSeats,PaidEMSSeats,PaidM365Seats,PaidOfficeSeats,HasUsGovCloudOffer,FirstPaidEXOStartDate,FirstPaidSPOStartDate,FirstPaidOD4BStartDate,FirstPaidSfBStartDate,FirstPaidYammerStartDate,FirstPaidTeamsStartDate,FirstPaidProPlusStartDate,FirstPaidAADPStartDate,FirstPaidAIPStartDate,FirstPaidAATPStartDate,FirstPaidIntuneStartDate,FirstPaidMCASStartDate,FirstPaidO365E5SkuStartDate,FirstPaidM365E5SkuStartDate,FirstPaidEMSE5SkuStartDate,Has1YearFreeDomain,Age,AU_exchange_04,AU_sharepoint_04,AU_skype_04,AU_teams_04,AU_od4b_04,AU_onenote_04,AU_word_04,AU_excel_04,AU_powerpoint_04,AU_outlook_04,AU_oatp_04,AU_odsp_04,AU_eslt_04,AU_officeclient_04,AU_exchange_05,AU_sharepoint_05,AU_skype_05,AU_teams_05,AU_od4b_05,AU_onenote_05,AU_word_05,AU_excel_05,AU_powerpoint_05,AU_outlook_05,AU_oatp_05,AU_odsp_05,AU_eslt_05,AU_officeclient_05,AU_exchange,AU_sharepoint,AU_skype,AU_teams,AU_od4b,AU_onenote,AU_word,AU_excel,AU_powerpoint,AU_outlook,AU_oatp,AU_odsp,AU_eslt,AU_officeclient,AR_exchange_04,AR_sharepoint_04,AR_skype_04,AR_teams_04,AR_od4b_04,AR_onenote_04,AR_word_04,AR_excel_04,AR_powerpoint_04,AR_outlook_04,AR_eslt_04,AR_officeclient_04,AR_exchange_05,AR_sharepoint_05,AR_skype_05,AR_teams_05,AR_od4b_05,AR_onenote_05,AR_word_05,AR_excel_05,AR_powerpoint_05,AR_outlook_05,AR_eslt_05,AR_officeclient_05,AR_exchange_06,AR_sharepoint_06,AR_skype_06,AR_teams_06,AR_od4b_06,AR_onenote_06,AR_word_06,AR_excel_06,AR_powerpoint_06,AR_outlook_06,AR_eslt_06,AR_officeclient_06
0,946A2663-3AD0-46B8-90C0-DE373075DCCE,2/5/2014 5:31:19 AM,11/14/2018 12:00:00 AM,BE,5,1,0,1,0,5,0,0,2,0,0,2,en,Global|GoLocal,PublicCloud|GoLocal,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,Fa

In [39]:
df.columns.to_list()

['TenantId',
 'CreatedDate',
 'CreateDateOfFirstSubscription',
 'CountryCode',
 'AllupSeats',
 'EXOSubscriptionsCount',
 'OD4BSubscriptionsCount',
 'SfBSubscriptionsCount',
 'TeamsSubscriptionsCount',
 'PaidCount',
 'ProjectSubscriptionsCount',
 'SPOSubscriptionsCount',
 'ActivatedSubscriptionTotalCount',
 'VisioSubscriptionsCount',
 'TrialSubscriptionsCount',
 'NonTrialSubscriptionsCount',
 'Languange',
 'DataCenterInstance',
 'DataCenterModel',
 'HasEXO',
 'HasSPO',
 'HasOD4B',
 'HasSfB',
 'HasYammer',
 'HasTeams',
 'HasTeamsFreemium',
 'HasKaizala',
 'HasProPlus',
 'HasAADP',
 'HasAIP',
 'HasAATP',
 'HasIntune',
 'HasMCAS',
 'HasWDATP',
 'HasAudioConference',
 'HasPhoneSystem',
 'HasEdiscovery',
 'HasCompliance',
 'HasThreatIntelligence',
 'HasCustomerLockbox',
 'HasOATP',
 'HasAADPP2',
 'HasAIPP2',
 'HasWindows',
 'HasO365CAS',
 'HasCASDiscovery',
 'HasPAM',
 'HasPowerBI',
 'HasPowerBIPremium',
 'HasPowerBIPro',
 'HasVisio',
 'HasProject',
 'HasNonTrial',
 'HasSubscription_AllCount

# Preprocess data

1. encode categorical data
2. transfer datetime data
3. convert nan to zero, inf to one
4. encode boolean type data
5. split dataset
6. normalize data

In [15]:
cols_name = pd.Series(data=df.columns)

ar_04_beg_col_index = cols_name[cols_name == 'AR_exchange_04'].index[0]
ar_06_beg_col_index = cols_name[cols_name == 'AR_exchange_06'].index[0]
ar_06_end_col_index = cols_name[cols_name == 'AR_officeclient_06'].index[0]

wl_AR_cols = cols_name[ar_04_beg_col_index : ar_06_end_col_index+1].tolist()

In [16]:
au_06_beg_col_index = cols_name[cols_name == 'AU_exchange'].index[0]
au_06_end_col_index = cols_name[cols_name == 'AU_officeclient'].index[0]

au_06_cols = cols_name[au_06_beg_col_index : au_06_end_col_index+1].tolist()

In [17]:
output_cols = cols_name[ar_06_beg_col_index : ar_06_end_col_index+1].tolist()


In [18]:
len(output_cols)
# len(wl_AR_cols)

12

In [19]:
len(au_06_cols)

14

In [6]:
def split_dataset(df, age=360):
    df_train = df.loc[df['Age'] >= age]
    df_test = df.loc[df['Age'] < age]
    
    df_train = df_train.drop('Age', axis=1)
    df_test = df_test.drop('Age', axis=1)
    
    print('df_test shape is {}'.format(df_test.shape))
    
    ### split dev set from training set ###
    dev_size = int(df_train.shape[0] * 0.01)

    df_dev = df_train.iloc[-dev_size:,:]
    df_train = df_train.iloc[:-dev_size,:]
    print('df_dev shape is {}'.format(df_dev.shape))
    print('df_train shape is {}'.format(df_train.shape))
    
    return df_train, df_dev, df_test

In [7]:
def separate_input_output_cols(df, predict_ahead=False):
    cols_name = pd.Series(data=df.columns)

    ar_04_beg_col_index = cols_name[cols_name == 'AR_exchange_04'].index[0]
    ar_06_beg_col_index = cols_name[cols_name == 'AR_exchange_06'].index[0]
    ar_06_end_col_index = cols_name[cols_name == 'AR_officeclient_06'].index[0]
    
    au_06_beg_col_index = cols_name[cols_name == 'AU_exchange'].index[0]
    au_06_end_col_index = cols_name[cols_name == 'AU_officeclient'].index[0]
    
    au_06_cols = cols_name[au_06_beg_col_index : au_06_end_col_index+1].tolist()

    wl_AR_cols = cols_name[ar_04_beg_col_index:ar_06_end_col_index+1].tolist()

    output_cols = cols_name[ar_06_beg_col_index:ar_06_end_col_index+1].tolist()

    df_y = df.loc[:, output_cols]
    print('outputs shape is {}'.format(df_y.shape))

    ### use profile info only, exclude the usage of previous months ###
    df_X = df.drop(columns=wl_AR_cols) 
    
    ### if we predict the usage at the beginning of the month, we should ###
    ### exclude the active users' number for that month (June).          ###
    if predict_ahead:
        df_X = df_X.drop(columns=au_06_cols) 
    
    print('inputs shape is {}'.format(df_X.shape))

    return df_X, df_y

In [8]:
def process_object_cols_for_inputs(df):
    ### only selected the StartDate for outputs workloads ###
    cols_datetime = ['CreatedDate', 'CreateDateOfFirstSubscription','FirstPaidEXOStartDate',
       'FirstPaidSPOStartDate', 'FirstPaidOD4BStartDate',
       'FirstPaidSfBStartDate', #'FirstPaidYammerStartDate',
       'FirstPaidTeamsStartDate', 'FirstPaidProPlusStartDate',
       #'FirstPaidAADPStartDate', 'FirstPaidAIPStartDate',
       #'FirstPaidAATPStartDate', 'FirstPaidIntuneStartDate',
       #'FirstPaidMCASStartDate', 'FirstPaidO365E5SkuStartDate',
       #'FirstPaidM365E5SkuStartDate', 'FirstPaidEMSE5SkuStartDate'
                    ]
    df_datetime = df.loc[:, cols_datetime]
    
    ### drop some redundant columns ###
    cols_cat = ['CountryCode', 'Languange', #'DataCenterInstance', 'DataCenterModel',
       #'SignupLocationInfo_Country', #'SignupLocationInfo_CountryCode',
       #'SignupLocationInfo_Region', 'TopParents_AreaName',
       #'TopParents_CountryCode', #'TopParents_BigAreaName', 
       'TopParents_Industry', #'TopParents_RegionName',
       #'TopParents_SegmentGroup', #'TopParents_SubRegionName',
       #'TopParents_VerticalName'
               ]
    df_cat = df.loc[:, cols_cat]
    
    df_tenantid = df.loc[:,'TenantId']
    
    return df_tenantid, df_cat, df_datetime

In [9]:
def encode_datetime(df_datetime):
    """Encode the datetime inputs from '2/5/2014 5:31:19 AM' format
        to a numerical number of UTC format.
        
    Args:
      df_datetime: a DataFrame that only stores the datetime inputs.
        
    Returns:
      X_datetime: a numpy array that contains the encoded datetime inputs.
      datetime_cols: a list that contains the datetime colunms name.
      
    """
    
    cols = df_datetime.columns
    for i in cols:
        df_datetime[i] = pd.to_datetime(df_datetime[i], utc=True,
                                        errors='coerce').astype(int,errors='ignore')
#     print(df_datetime.head())
        
    X_datetime = df_datetime.to_numpy()
    datetime_cols = cols.to_list()
    
    return X_datetime, datetime_cols

In [10]:
def encode_num_bool(df):
    """Encode the numerical and boolean inputs to an array.
        
    Args:
      df: a DataFrame that stores the inputs raw data.
        
    Returns:
      X_bool: a numpy array that contains the encoded boolean inputs.
      X_num: a numpy array that contains the encoded numerical inputs.
      num_cols: a list that contains the numerical columns name.
      bool_cols: a list that contains the boolean columns name.
    """
    
    X_num = df.select_dtypes(include=['float','int'])
    X_bool = df.select_dtypes(include='bool')

    num_cols = X_num.columns.to_list()
    bool_cols = X_bool.columns.to_list()
    
    X_bool = X_bool.astype(int).to_numpy()
    X_num = X_num.to_numpy()
    return X_bool, X_num, num_cols, bool_cols

In [16]:
def encode_training_inputs(df_X):
    """Encode the inputs for training set.
        
    Args:
      df_X: a DataFrame that stores the inputs raw data.
        
    Returns:
      X_scal: a numpy array that contains the encoded and normalized inputs.
      dv: a DictVectorizer that is trained on the training set.
      scaler: a StandardScaler that is trained on the training set.
      cols_name: a list that contains all of the inputs features after encoding.
      df_X_id: a DataFrame that contains the TenantId for training set.
    """
    
    
    print('Starting to encode training inputs:')
    X_bool, X_num, num_cols, bool_cols = encode_num_bool(df_X)
    
    df_X_id, df_X_cat, df_X_datetime = process_object_cols_for_inputs(df_X)
    
    X_datetime, datetime_cols = encode_datetime(df_X_datetime)
    
#     id_file_name = df_X_id.name + '_train.csv'
#     df_X_id.to_csv(id_file_name, header=False)
    
    ### encode the categorical columns ###
    X_cat_dict = df_X_cat.to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_cat_encoded = dv.fit_transform(X_cat_dict)
    vocab = dv.vocabulary_
    vocab_od = collections.OrderedDict(sorted(vocab.items(), key=lambda x:x[1]))
    cat_encoded_cols = list(vocab_od.keys())
#     print(X_cat_encoded)
    
    ### normalize all the inputs ###
    X_arr = np.concatenate((X_cat_encoded, X_num, X_bool, X_datetime), axis=1)
    scaler = StandardScaler()
    X_scal = scaler.fit_transform(X_arr)
#     print(X_arr.shape)
    
    cols_name = cat_encoded_cols + num_cols + bool_cols + datetime_cols

    assert len(cols_name) == X_scal.shape[1]
    
    return X_scal, dv, scaler, cols_name, df_X_id

In [17]:
def encode_dev_test_inputs(df_X, dv, scaler):
    """Encode the inputs for dev or test set.
        
    Args:
      df_X: a DataFrame that stores the inputs raw data.
      dv: a DictVectorizer that is trained on the training set.
      scaler: a StandardScaler that is trained on the training set.
        
    Returns:
      X_scal: a numpy array that contains the encoded and normalized inputs.
      df_X_id: a DataFrame that contains the TenantId for dev or test set.
    """
    
    print('Starting to encode dev or test inputs:')
    X_bool, X_num, _, _ = encode_num_bool(df_X)
    df_X_id, df_X_cat, df_X_datetime = process_object_cols_for_inputs(df_X)
    
#     id_file_name = df_X_id.name + '_' + dataset_type + '.csv'
#     df_X_id.to_csv(id_file_name, header=False)
    
    X_datetime, _ = encode_datetime(df_X_datetime)
    
    ### encode the categorical columns ###
    X_cat_dict = df_X_cat.to_dict(orient='records')
    X_cat_encoded = dv.transform(X_cat_dict)
#     print(X_cat_encoded.shape)
    
    ### normalize all the inputs ###
    X_arr = np.concatenate((X_cat_encoded, X_num, X_bool, X_datetime), axis=1)
#     print(X_arr.shape)
    
    X_scal = scaler.transform(X_arr)
    
    return X_scal, df_X_id

In [36]:
import os
a = 'model'

model_path = os.path.join('/data/home/t-chepan/projects/MS-intern-project/results', '{}.json'.format(a))

model_path

'/data/home/t-chepan/projects/MS-intern-project/results/model.json'

In [23]:
df_train, df_dev, df_test = split_dataset(df)
df_train_X, df_train_y = separate_input_output_cols(df_train)
df_dev_X, df_dev_y = separate_input_output_cols(df_dev)
df_test_X, df_test_y = separate_input_output_cols(df_test)

df_test shape is (260, 227)
df_dev shape is (7, 227)
df_train shape is (733, 227)
outputs shape is (733, 12)
inputs shape is (733, 191)
outputs shape is (7, 12)
inputs shape is (7, 191)
outputs shape is (260, 12)
inputs shape is (260, 191)


In [24]:
Xtrain, dv, scaler, cols_name, df_train_id = encode_training_inputs(df_train_X)
Xdev, df_dev_id = encode_dev_test_inputs(df_dev_X, dv, scaler)
Xtest, df_test_id = encode_dev_test_inputs(df_test_X, dv, scaler)

print('After encoding, the training size is {}'.format(Xtrain.shape))
print('After encoding, the dev size is {}'.format(Xdev.shape))
print('After encoding, the test size is {}'.format(Xtest.shape))


Starting to encode training inputs:
Starting to encode dev or test inputs:
Starting to encode dev or test inputs:
After encoding, the training size is (733, 418)
After encoding, the dev size is (7, 418)
After encoding, the test size is (260, 418)


In [25]:
ytrain = df_train_y.to_numpy()
ydev = df_dev_y.to_numpy()
ytest = df_test_y.to_numpy()
print('Outputs are ready!')

np.save('data/ytrain_fake.npy', ytrain)
np.save('data/ydev_fake.npy', ydev)
np.save('data/ytest_fake.npy', ytest)

print('Saved the outputs targets!')

Outputs are ready!
Saved the outputs targets!


In [26]:
# with open('encoded_columns.txt', 'w') as f:
#     for item in cols_name:
#         f.write("%s\n" % item)
       
# joblib.dump(dv, 'results/vectorizer.pkl')

In [27]:
np.save('data/Xtrain_fake.npy', Xtrain)
np.save('data/Xdev_fake.npy', Xdev)
np.save('data/Xtest_fake.npy', Xtest)
print('Saved the encoded inputs!')

Saved the encoded inputs!


In [32]:
a = 0

if a:
    a = 100

a

0